Data referenced below:

Task C., Bhagat K., Streat D., Simpson A., and Howarth G.S. (2023),
NIST Diverse Communities Data Excerpts, National Institute of Standards and Technology,
https://doi.org/10.18434/mds2-2895

In [323]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0


We want to clean this data for use in the gradient descent algorithm
- Predictors consist of things that you might be able to discern from a quick look at someone on the street
    - AGEP (age), SEX, RAC1P (Race separated into different categories), DPHY (ambulatory disability e.g. wheelchair or limp), DEYE (visual impairment)
- The idea is to train a model to predict somebodies invisible traits such as income, education, marital status, etc. based on their visible ones.
    - As a proof of concept that this is possible with the NIST data, we will be trying to predict whether someone is above/below the poverty line (using logistic regression) and what income decile they are in (using linear regression)

In [324]:
# Load data files
import urllib.request
import io

# import nation data from NIST
nist_national_2019 = pd.read_csv('https://media.githubusercontent.com/media/usnistgov/SDNist/refs/heads/main/nist%20diverse%20communities%20data%20excerpts/national/national2019.csv')

# remove NA values (represented as 'N' in this dataset)
nist_national_2019 = nist_national_2019[nist_national_2019["AGEP"] != "N"]
nist_national_2019 = nist_national_2019[nist_national_2019["RAC1P"] != "N"]
nist_national_2019 = nist_national_2019[nist_national_2019["SEX"] != "N"]
nist_national_2019 = nist_national_2019[nist_national_2019["DPHY"] != "N"]
nist_national_2019 = nist_national_2019[nist_national_2019["POVPIP"] != "N"]
nist_national_2019 = nist_national_2019[nist_national_2019["PINCP_DECILE"] != "N"]

# scale the data to be within (-1 - 1)
nist_national_2019["SEX"] = (nist_national_2019["SEX"] - 1) * 2 - 1
nist_national_2019["AGEP"] = nist_national_2019["AGEP"]/99 # we can do this since AGEP has set bounds in the data_dictionary
for i in range(9):
    # each category of race is given its own column and a value of 1 is applied to the column that represents that race category
    nist_national_2019["RAC1P-" + str(i+1)] = nist_national_2019['RAC1P'].apply(lambda x: 1 if x == (i+1) else -1)
# -1 if a person is not hispanic 0 otherwise
nist_national_2019["HISP"] = nist_national_2019['HISP'].apply(lambda x: -1 if x == 0 else 1)
# 1 if a person has an ocular disability, -1 otherwise
nist_national_2019["DEYE"] = nist_national_2019['DEYE'].apply(lambda x: 1 if x == 1 else -1)
# 1 if a person has a physical disability, -1 otherwise
nist_national_2019["DPHY"] = pd.to_numeric(nist_national_2019["DPHY"]).apply(lambda x: 1 if x == 1 else -1)
# 1 if a record is (at or) below the poverty line
nist_national_2019["POVPIP"] = pd.to_numeric(nist_national_2019["POVPIP"]).apply(lambda x: 1 if x <= 100 else -1)

For our first set of tests, we will be performing Logistic Regression, trying to predict whether someone is above/below the poverty line.

In [325]:
X = nist_national_2019[["AGEP", "SEX", "HISP", "RAC1P-1", "RAC1P-2", "RAC1P-3", "RAC1P-4", "RAC1P-5", "RAC1P-6", "RAC1P-7", "RAC1P-8", "RAC1P-9", "DPHY", "DEYE"]]
y = nist_national_2019[["POVPIP"]]

y = np.ravel(np.array(y, dtype=float))
X = np.array(X)

# print(y.shape)
# print(X.shape)

# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 17268 4318


**Sci-Kit Learn Implementation**
This is a pre-built implementation of Logistic Regression that does not satisfy differential privacy.
We can use this as a benchmark for our own implementations.

In [326]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def train_model():
    model = LogisticRegression(max_iter=2000)
    model.fit(X_train, y_train)
    return model
model = train_model()
# print('Model coefficients:', model.coef_[0])
print('Model accuracy:', np.sum(model.predict(X_test) == y_test)/X_test.shape[0])

Model accuracy: 0.89601667438629


We can see that the sci-kit learn model has very good (and very fast) results.
But what about if we wanted to create a differentially private model to predict whether someone is below the poverty line?

First we need a few functions to allow us to perform both linear and logistic regression

In [327]:

# The loss function measures how good our model is. The training goal is to minimize the loss.

# This is the logistic loss function.
def logistic_loss(theta, xi, yi):
    exponent = - yi * (xi.dot(theta))
    return np.log(1 + np.exp(exponent))

# Mean Square Error for use in linear regression
def linear_loss(theta, xi, yi):
    return np.mean((yi - xi) ** 2)

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def logistic_gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

# Mean Square Error gradient for use in linear regression
def linear_gradient(theta, X, y):
    N = X.shape[0]
    y = np.array(y, dtype=float)
    predictions = X @ theta
    residuals = y - predictions
    return -2 / N * X.T @ residuals

# function to get a single prediction for a model based on an input
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias)
    return label

def linear_predict(xi, theta, bias=0):
    label = ((xi @ theta + bias).astype(int)).clip(min=0, max=9)
    # print(label)
    return label

# accuracy heuristic for comparing our different algorithms
def accuracy(theta):
    np.set_printoptions(threshold=np.inf)
    # print(predict(X_test, theta))
    # print(y_test)
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

def linear_accuracy(theta):
    # print(y_test)
    return np.sum(linear_predict(X_test, theta) == y_test.astype(int))/X_test.shape[0]

The following implementation does not satisfy differential privacy, but it's also another good baseline for us to compare our differentially private models to.

In [328]:
# function to get the average gradients in a model. Only used in
def avg_grad(theta, X, y, gradient):
    assert len(X) == len(y)
    gradients = [gradient(theta, X[i], y[i]) for i in range(len(X))]

    avg_gradient = np.mean(gradients, axis=0)
    return avg_gradient

# gradient descent algorithm. runs for a set number of iterations, and allows the user to specify which loss function to use (logistic/linear)
def gradient_descent(iterations, gradient):
    theta = [0 for _ in range(X_test.shape[1])]
    for iteration in range(iterations):
        theta = theta - avg_grad(theta, X_train, y_train, gradient)
    return theta

theta = gradient_descent(200, logistic_gradient)
print("Baseline Logistic Regression Accuracy: " + str(accuracy(theta)))

Baseline Logistic Regression Accuracy: 0.8967114404817045


We can now modify the gradient descent algorithm that we used previously to add noise to the gradients in each training iteration.
There's a few questions:
- How much privacy cost should we allocate when adding noise?
- How much noise do we have to add to satisfy epsilon-delta differential privacy?

The main issue is with the sensitivity of the gaussian_mech_vec. We can call the L2_clip() function on each gradient to clip it to a constant value. This allows us to control the sensitivity of the gaussian_mech_vec() function, as it is the same as whatever value we choose to clip the gradients to.
In the code below that value can be controlled by the L2_clip_param argument in the noisy_gradient_descent() function

In [329]:
def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)

    if norm > b:
        return b * (v / norm)
    else:
        return v

def noisy_gradient_descent(iterations, epsilon, delta, L2_clip_param, gradient):
    theta = np.array([0 for _ in range(X_test.shape[1])])
    for iteration in range(iterations):
        gradients = [gradient(theta, X_train[i], y_train[i]) for i in range(len(X_train))]
        clipped_grads = [L2_clip(g, L2_clip_param) for g in gradients]
        grad_sum = np.sum(clipped_grads, axis=0)

        noisy_grad_sum = gaussian_mech_vec(grad_sum, sensitivity=L2_clip_param, epsilon= (epsilon / iterations), delta = (delta/iterations))
        # reveals size of training data
        noisy_grad = np.array(noisy_grad_sum) / len(X_train)
        theta = theta - noisy_grad
    return theta


In standard stochastic gradient descent, we use every piece of training data in every iteration of training.
The main difference here in the mini-batch algorithm is that we only train on a subset of the training data (randomly chosen). This allows us to train more quickly due to the smaller iteration sizes, but still hopefully get similar results.

In [330]:
def noisy_gradient_descent_batching(iterations, epsilon, delta, L2_clip_param, batch_size, gradient):
    theta = np.array([0 for _ in range(X_test.shape[1])])
    for iteration in range(iterations):
        batch_indices = []
        # build the batch indices using random sampling
        while(len(batch_indices) != batch_size):
            rand = np.random.randint(0, len(X_train))
            if rand not in batch_indices:
                batch_indices.append(rand)
        gradients = [gradient(theta, X_train[i], y_train[i]) for i in batch_indices]
        clipped_grads = [L2_clip(g, L2_clip_param) for g in gradients] #L2 sensitivity is 5
        grad_sum = np.sum(clipped_grads, axis=0)

        noisy_grad_sum = gaussian_mech_vec(grad_sum, sensitivity=L2_clip_param, epsilon= (epsilon / iterations), delta = (delta/iterations))
        # reveals size of training data
        noisy_grad = np.array(noisy_grad_sum) / len(X_train)
        theta = theta - noisy_grad
    return theta

In [331]:
s_gd = time.time()
theta = noisy_gradient_descent(50, 1.0, 1e-5, 5, logistic_gradient)
e_gd = time.time()

s_bgd = time.time()
theta_prime = noisy_gradient_descent_batching(50, 1.0, 1e-5, 5, int(X_train.shape[0]/10), logistic_gradient)
e_bgd = time.time()
print('Noisy Gradient Descent Accuracy:', accuracy(theta))
print('Noisy Gradient Descent Time:', e_gd-s_gd)
print('Noisy Mini-batch Descent Accuracy:', accuracy(theta_prime))
print('Noisy Mini-batch Descent Time:', e_bgd-s_bgd)

Noisy Gradient Descent Accuracy: 0.8471514590088004
Noisy Gradient Descent Time: 3.001326084136963
Noisy Mini-batch Descent Accuracy: 0.8967114404817045
Noisy Mini-batch Descent Time: 0.8987910747528076


We were able to successfully predict whether someone was above the poverty line based on several (visible) columns.
It would also be interesting to be able to predict something in a non-binary manner. Such as the decile of income of a person.

Below we set the y (prediction) set to be the Principle Decile column from the NIST dataset, and then split up the data into training/test data.

In [332]:
# Load data files

X = nist_national_2019[["AGEP", "SEX", "HISP", "RAC1P-1", "RAC1P-2", "RAC1P-3", "RAC1P-4", "RAC1P-5", "RAC1P-6", "RAC1P-7", "RAC1P-8", "RAC1P-9", "DPHY", "DEYE"]]
y = nist_national_2019[["PINCP_DECILE"]]

y = np.ravel(np.array(y))
X = np.array(X)

# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 17268 4318


In [333]:
def noisy_gradient_descent_linear(iterations, epsilon, delta, L2_clip_param, gradient):
    theta = np.array([0 for _ in range(X_test.shape[1])])
    for iteration in range(iterations):
        gradients = gradient(theta, X_train, y_train)
        clipped_grad = L2_clip(gradients, L2_clip_param)
        noisy_grad = gaussian_mech(clipped_grad, sensitivity=L2_clip_param, epsilon= (epsilon / iterations), delta = (delta/iterations))
        theta = theta - noisy_grad
    return theta

In [334]:
def noisy_gradient_descent_batching_linear(iterations, epsilon, delta, L2_clip_param, batch_size, gradient):
    theta = np.array([0 for _ in range(X_test.shape[1])])

    for iteration in range(iterations):
        batch_indices = []
        # build the batch indices using random sampling
        while(len(batch_indices) != batch_size):
            rand = np.random.randint(0, len(X_train))
            if rand not in batch_indices:
                batch_indices.append(rand)
        X_set = np.array([X_train[i] for i in batch_indices])
        y_set = np.array([y_train[i] for i in batch_indices])
        gradients = gradient(theta, X_set, y_set)
        clipped_grad = L2_clip(gradients, L2_clip_param)
        noisy_grad = gaussian_mech(clipped_grad, sensitivity=L2_clip_param, epsilon= (epsilon / iterations), delta = (delta/iterations))
        theta = theta - noisy_grad
    return theta

In [336]:
s_gd = time.time()
theta = noisy_gradient_descent_linear(500, 1.0, 1e-5, 5, linear_gradient)
e_gd = time.time()

s_bgd = time.time()
theta_prime = noisy_gradient_descent_batching_linear(500, 1.0, 1e-5, 5, int(X_train.shape[0]/100), linear_gradient)
e_bgd = time.time()
print('Noisy Gradient Descent Accuracy:', linear_accuracy(theta))
print('Noisy Gradient Descent Time:', e_gd-s_gd)
print('Noisy Mini-batch Descent Accuracy:', linear_accuracy(theta_prime))
print('Noisy Mini-batch Descent Time:', e_bgd-s_bgd)

Noisy Gradient Descent Accuracy: 0.182260305697082
Noisy Gradient Descent Time: 0.7623159885406494
Noisy Mini-batch Descent Accuracy: 0.1000463177396943
Noisy Mini-batch Descent Time: 0.20416569709777832


No matter how we change our parameters, it seems like linear regression is not able to find a solution, and generally ends with <25% accuracy. This is probably due to several factors:
1. We are trying to predict a column with 10 different possibilities using only 14 predictors. This is a fairly complex problem
2. A persons income decile is a complex issue that is not easily able to be determined. We had much more success with the poverty line question (using logistic regression) because there are only two possibilities; a person is either above or below the poverty line.

The added complexity of predicting a persons income decile is too complex to predict just based on their visual predictors. Perhaps adding more predictors would make this more feasible.